Train MLP for each finger pair!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne
from random import shuffle
import random

mne.set_log_level('error')


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
# Set seed for NumPy
np.random.seed(42)

# Set seed for Python's built-in random number generator
random.seed(42)

# Set seed for PyTorch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
subject_id = 0
tag = 'gpt'

In [4]:
# Load the dictionary from the HDF5 file
target_dir = 'features'

file_path = os.path.join(target_dir, tag+'_'+cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    data[key] = np.reshape(value, (value.shape[0], -1))
    print(key, data[key].shape)

index (50, 8568)
little (50, 8568)
middle (50, 8568)
ring (50, 8568)
thumb (50, 8568)


In [5]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [6]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test, y_pred)
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 2, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(finger1, finger2, verbose = True):
   
    print(f'Training MLP for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)

   
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=10)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')
    return best_trial.value



In [7]:
train_MLP('middle', 'ring', verbose = True)

[I 2023-04-15 14:29:59,100] A new study created in memory with name: no-name-04364c1a-568a-4e29-9d7a-57d10ab2ca81
[I 2023-04-15 14:29:59,295] Trial 0 finished with value: 0.8 and parameters: {'learning_rate': 0.0013178216079708042, 'num_epochs': 117, 'hidden_size': 40, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.8.


Training MLP for middle vs ring


[I 2023-04-15 14:30:03,449] Trial 1 finished with value: 0.75 and parameters: {'learning_rate': 1.0767283185969204e-05, 'num_epochs': 1060, 'hidden_size': 41, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:30:05,575] Trial 2 finished with value: 0.75 and parameters: {'learning_rate': 0.0017155970725842638, 'num_epochs': 428, 'hidden_size': 61, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:30:10,260] Trial 3 finished with value: 0.75 and parameters: {'learning_rate': 0.00011330360933387461, 'num_epochs': 1200, 'hidden_size': 118, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:30:15,965] Trial 4 finished with value: 0.75 and parameters: {'learning_rate': 0.003030714938558828, 'num_epochs': 1515, 'hidden_size': 115, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:30:17,799] Trial 5 finished with value: 0.75 a

Best trial params: {'learning_rate': 0.0013178216079708042, 'num_epochs': 117, 'hidden_size': 40, 'activation': 'elu', 'optimizer': 'SGD'}
Best trial accuracy: 80.00%


0.8

In [8]:
accuracies = []
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            acc = train_MLP(finger1, finger2)
            accuracies.append(acc)
        else:
            break

print("#" * 50)
print(f'Mean accuracy: {np.mean(accuracies) * 100:.2f}%')
print(f'Std accuracy: {np.std(accuracies) * 100:.2f}%')
print("#" * 50)

[I 2023-04-15 14:30:42,635] A new study created in memory with name: no-name-48d855a1-8959-46be-90b2-375501b2b385


Training MLP for little vs index


[I 2023-04-15 14:30:43,594] Trial 0 finished with value: 0.4 and parameters: {'learning_rate': 0.0007417871658998917, 'num_epochs': 464, 'hidden_size': 47, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:30:49,123] Trial 1 finished with value: 0.4 and parameters: {'learning_rate': 0.047400417968021274, 'num_epochs': 628, 'hidden_size': 114, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:30:51,929] Trial 2 finished with value: 0.35 and parameters: {'learning_rate': 0.0001537204331438265, 'num_epochs': 920, 'hidden_size': 86, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:30:59,916] Trial 3 finished with value: 0.35 and parameters: {'learning_rate': 0.0005639329492607437, 'num_epochs': 1088, 'hidden_size': 114, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:31:06,380] Trial 4 finished with value

Best trial params: {'learning_rate': 5.518492987235711e-05, 'num_epochs': 903, 'hidden_size': 102, 'activation': 'leaky_relu', 'optimizer': 'SGD'}
Best trial accuracy: 45.00%
Training MLP for middle vs index


[I 2023-04-15 14:31:30,379] Trial 0 finished with value: 0.4 and parameters: {'learning_rate': 2.5777990720492858e-05, 'num_epochs': 1929, 'hidden_size': 58, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:31:32,451] Trial 1 finished with value: 0.4 and parameters: {'learning_rate': 0.0037800524199333714, 'num_epochs': 622, 'hidden_size': 105, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4.
[I 2023-04-15 14:31:38,407] Trial 2 finished with value: 0.5 and parameters: {'learning_rate': 3.407677375242087e-05, 'num_epochs': 803, 'hidden_size': 116, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 2 with value: 0.5.
[I 2023-04-15 14:31:43,316] Trial 3 finished with value: 0.55 and parameters: {'learning_rate': 0.060272410127170814, 'num_epochs': 1310, 'hidden_size': 116, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 3 with value: 0.55.
[I 2023-04-15 14:31:44,678] Trial 4 finished w

...

Best trial params: {'learning_rate': 0.007324051834800903, 'num_epochs': 364, 'hidden_size': 31, 'activation': 'leaky_relu', 'optimizer': 'Adam'}
Best trial accuracy: 70.00%
Training MLP for thumb vs middle


[I 2023-04-15 14:37:21,542] Trial 0 finished with value: 0.55 and parameters: {'learning_rate': 0.0028682422215850606, 'num_epochs': 834, 'hidden_size': 43, 'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.55.
[I 2023-04-15 14:37:23,158] Trial 1 finished with value: 0.7 and parameters: {'learning_rate': 0.00010736442737393576, 'num_epochs': 290, 'hidden_size': 72, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7.
[I 2023-04-15 14:37:25,721] Trial 2 finished with value: 0.6 and parameters: {'learning_rate': 0.00011780704322836489, 'num_epochs': 1971, 'hidden_size': 20, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7.
[I 2023-04-15 14:37:31,447] Trial 3 finished with value: 0.7 and parameters: {'learning_rate': 0.01941158931453243, 'num_epochs': 1337, 'hidden_size': 51, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7.
[I 2023-04-15 14:37:35,091] Trial 4 finished with value: 0.6

Best trial params: {'learning_rate': 0.00010736442737393576, 'num_epochs': 290, 'hidden_size': 72, 'activation': 'relu', 'optimizer': 'Adam'}
Best trial accuracy: 70.00%
Training MLP for thumb vs ring


[I 2023-04-15 14:38:05,625] Trial 0 finished with value: 0.8 and parameters: {'learning_rate': 1.3538425121950146e-05, 'num_epochs': 1321, 'hidden_size': 70, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:38:14,090] Trial 1 finished with value: 0.75 and parameters: {'learning_rate': 0.01569869295000378, 'num_epochs': 897, 'hidden_size': 73, 'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:38:17,625] Trial 2 finished with value: 0.75 and parameters: {'learning_rate': 0.07524116558554925, 'num_epochs': 1366, 'hidden_size': 27, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8.
[I 2023-04-15 14:38:30,241] Trial 3 finished with value: 0.85 and parameters: {'learning_rate': 0.00016274700602542693, 'num_epochs': 1752, 'hidden_size': 101, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 3 with value: 0.85.
[I 2023-04-15 14:38:31,036] Trial 4 finished with value: 0.75 an

Best trial params: {'learning_rate': 0.00013678107258986296, 'num_epochs': 1491, 'hidden_size': 62, 'activation': 'relu', 'optimizer': 'SGD'}
Best trial accuracy: 90.00%
##################################################
Mean accuracy: 70.00%
Std accuracy: 12.04%
##################################################
